# 데이터 명세

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random

# 한글 깨짐 방지
import koreanize_matplotlib

#Time Series Analysis
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller, kpss, ccf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# clustering
from sklearn.cluster import KMeans

# 스케일링
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# 모델링
from pycaret.classification import *
from sklearn.model_selection import cross_validate
from keras.models import Sequential
from tensorflow import keras
from keras.layers import LSTM
from keras.layers import Dense, Dropout
from sklearn.metrics import mean_absolute_error
from statsmodels.tsa.arima.model import ARIMA

# 시스템
from tqdm.notebook import tqdm
import os
import warnings
warnings.filterwarnings('ignore')

2023-11-07 01:42:50.385357: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-07 01:42:50.426370: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-07 01:42:50.426422: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-07 01:42:50.426457: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-07 01:42:50.433169: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-07 01:42:50.434060: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [3]:
train = pd.read_csv('casting.csv', encoding = 'cp949')
train

,Unnamed: 0,line,name,mold_name,time,date,count,working,emergency_stop,molten_temp,...,lower_mold_temp3,sleeve_temperature,physical_strength,Coolant_temperature,EMS_operation_time,registration_time,passorfail,tryshot_signal,mold_code,heating_furnace
0,0,전자교반 3라인 2호기,TM Carrier RH,TM Carrier RH-Semi-Solid DIE-06,2019-01-02,16:45:06,258,가동,ON,731.0,...,1449.0,550.0,700.0,34.0,23,2019-01-02 16:45:06,0.0,NaN,8722,NaN
1,1,전자교반 3라인 2호기,TM Carrier RH,TM Carrier RH-Semi-Solid DIE-06,2019-01-02,16:45:08,243,가동,ON,720.0,...,NaN,481.0,0.0,30.0,25,2019-01-02 16:45:08,0.0,NaN,8412,B
2,2,전자교반 3라인 2호기,TM Carrier RH,TM Carrier RH-Semi-Solid DIE-06,2019-01-02,16:45:58,244,가동,ON,721.0,...,NaN,481.0,0.0,30.0,25,2019-01-02 16:45:58,0.0,NaN,8412,B
3,3,전자교반 3라인 2호기,TM Carrier RH,TM Carrier RH-Semi-Solid DIE-06,2019-01-02,16:48:03,245,가동,ON,721.0,...,NaN,483.0,0.0,30.0,25,2019-01-02 16:48:03,0.0,NaN,8412,B
4,4,전자교반 3라인 2호기,TM Carrier RH,TM Carrier RH-Semi-Solid DIE-06,2019-01-02,16:50:08,246,가동,ON,721.0,...,NaN,486.0,0.0,30.0,25,2019-01-02 16:50:08,0.0,NaN,8412,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92010,92010,전자교반 3라인 2호기,TM Carrier RH,TM Carrier RH-Semi-Solid DIE-06,2019-03-31,23:51:29,71,가동,ON,731.0,...,1449.0,279.0,726.0,30.0,0,2019-03-31 23:51:29,0.0,NaN,8917,A
92011,92011,전자교반 3라인 2호기,TM Carrier RH,TM Carrier RH-Semi-Solid DIE-06,2019-03-31,23:53:28,72,가동,ON,731.0,...,1449.0,279.0,732.0,30.0,0,2019-03-31 23:53:28,0.0,NaN,8917,A
92012,92012,전자교반 3라인 2호기,TM Carrier RH,TM Carrier RH-Semi-Solid DIE-06,2019-03-31,23:55:44,73,가동,ON,732.0,...,1449.0,272.0,731.0,30.0,0,2019-03-31 23:55:44,0.0,NaN,8917,A
92013,92013,전자교반 3라인 2호기,TM Carrier RH,TM Carrier RH-Semi-Solid DIE-06,2019-03-31,23:57:46,74,가동,ON,732.0,...,1449.0,288.0,723.0,30.0,0,2019-03-31 23:57:46,0.0,NaN,8917,A


In [4]:
train.drop('Unnamed: 0', axis = 1, inplace = True)
train.columns = ['작업라인', '제품명', '금형명', '수집날짜', '수집일시', '일자별 제품 생산 번호',
                '가동여부', '비상정지', '용탕온도', '설비 작동 사이클시간', '제품생산 사이클 시간',
                 '저속구간속도', '고속구간속도', '용탕량', '주조압력', '비스켓 두께', '상금형온도1',
                '상금형온도2', '상금형온도3', '하금형온도1', '하금형온도2', '하금형온도3', '슬리브온도',
                '형체력', '냉각수 온도', '전자교반 가동시간', '등록일시', '양품불량판정', '사탕신호',
                 '금형코드', '가열로']

In [5]:
# 작업라인x, 제품명x, 금형명x, 수집날짜x, 수집일시x, 

In [7]:
train_df = train.drop(['작업라인', '제품명', '금형명', '수집날짜', '수집일시',
                       '가동여부', '비상정지', '전자교반 가동시간', '등록일시'], axis = 1)

In [12]:
train_df = train_df.drop(train_df[train_df['저속구간속도'].isna()].index, axis = 0)

In [13]:
train_df.head()

,일자별 제품 생산 번호,용탕온도,설비 작동 사이클시간,제품생산 사이클 시간,저속구간속도,고속구간속도,용탕량,주조압력,비스켓 두께,상금형온도1,...,하금형온도1,하금형온도2,하금형온도3,슬리브온도,형체력,냉각수 온도,양품불량판정,사탕신호,금형코드,가열로
0,258,731.0,119,120,110.0,112.0,75.0,331.0,35.0,198.0,...,234.0,316.0,1449.0,550.0,700.0,34.0,0.0,NaN,8722,NaN
1,243,720.0,98,125,109.0,109.0,NaN,309.0,48.0,250.0,...,208.0,163.0,NaN,481.0,0.0,30.0,0.0,NaN,8412,B
2,244,721.0,98,122,109.0,109.0,NaN,309.0,48.0,250.0,...,208.0,163.0,NaN,481.0,0.0,30.0,0.0,NaN,8412,B
3,245,721.0,100,125,112.0,109.0,NaN,308.0,49.0,268.0,...,211.0,179.0,NaN,483.0,0.0,30.0,0.0,NaN,8412,B
4,246,721.0,99,123,109.0,110.0,NaN,309.0,49.0,276.0,...,216.0,187.0,NaN,486.0,0.0,30.0,0.0,NaN,8412,B


In [15]:
setup_clf = setup(train_df, target = '양품불량판정',
                  numeric_features = ['일자별 제품 생산 번호', '용탕온도', '설비 작동 사이클시간', '제품생산 사이클 시간',
                                     '저속구간속도', '고속구간속도', '용탕량', '주조압력', '비스켓 두께', '상금형온도1', '상금형온도2',
                                     '상금형온도3', '하금형온도1', '하금형온도2', '하금형온도3', '슬리브온도', '형체력', '냉각수 온도'],
                  categorical_features = ['사탕신호', '금형코드', '가열로'],
                  n_jobs = -1,
                  session_id = 42)

,Description,Value
0,Session id,42
1,Target,양품불량판정
2,Target type,Binary
3,Original data shape,"(92014, 22)"
4,Transformed data shape,"(92014, 28)"
5,Transformed train set shape,"(64409, 28)"
6,Transformed test set shape,"(27605, 28)"
7,Ordinal features,1
8,Numeric features,18
9,Categorical features,3


In [16]:
best = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.9916,0.9952,0.8520,0.9498,0.8980,0.8936,0.8952,0.9290
xgboost,Extreme Gradient Boosting,0.9916,0.9958,0.8691,0.9351,0.9006,0.8962,0.8970,0.1370
et,Extra Trees Classifier,0.9914,0.9953,0.8353,0.9627,0.8943,0.8899,0.8924,0.3840
lightgbm,Light Gradient Boosting Machine,0.9914,0.9956,0.8627,0.9366,0.8979,0.8935,0.8944,137.4090
gbc,Gradient Boosting Classifier,0.9895,0.9868,0.8161,0.9353,0.8714,0.8659,0.8683,1.2680
dt,Decision Tree Classifier,0.9885,0.9285,0.8627,0.8729,0.8676,0.8616,0.8617,0.1670
knn,K Neighbors Classifier,0.9881,0.9597,0.7677,0.9494,0.8487,0.8426,0.8478,0.5520
ada,Ada Boost Classifier,0.9859,0.9854,0.7467,0.9155,0.8220,0.8148,0.8196,0.3450
lr,Logistic Regression,0.9842,0.9425,0.6677,0.9584,0.7863,0.7784,0.7926,1.0120
lda,Linear Discriminant Analysis,0.9829,0.9329,0.6190,0.9818,0.7585,0.7501,0.7720,0.0800
